In [39]:
import matplotlib
matplotlib.use("Agg")  # Must be the FIRST line

import matplotlib.pyplot as plt


plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
})

# Confirm backend
print("Backend in use:", matplotlib.get_backend())  # must print 'agg'

# import modules
import numpy as np
import seaborn as sns; sns.set()

# import the sklearn modules                                                                                                               
import sklearn
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, RocCurveDisplay, make_scorer, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.model_selection import cross_val_score


import pickle
import pandas as pd

import os

Backend in use: agg


In [40]:
# Read the data

dfnoise = pd.read_csv('/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/data/O3a/noise_no_catalogue.csv', sep = ',')
dfinj = pd.read_csv('/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/data/O3a/injections.csv', sep = ',')
dfEvents_confident = pd.read_csv('/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/data/O3a-events/GWTCO3a.csv', delimiter = ',')

In [43]:
# Define the path and features

# Path specifics  
TYPE = "DOUBLE" # Type of coincidence
Ifo = 'HL' # Interferometers
F = 'F_ER' # Features
M = 'M_Best' # Type of model


#Define the features
features_inj = [#'index', 
                'L_snr', 'H_snr','V_snr',#F0
                'amplitude', #F9
                'L_autochi^2_PQ',	'H_autochi^2_PQ', #F0
                'm1', 'm2', #F1
                'mc',
                's1z', 's2z', #F2
                't_dur', #F3
                'nEvents', #F4
                'L_ERw', 'H_ERw', #F5
                'L_phase',	'H_phase', #F6
                'L_effDist', 'H_effDist', #F7
                'Lend_time', 'Hend_time',
                'iFAR',
                'label',
                'gps_time',
            ]

savefig_path = f'/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/O3a-results-figs/{Ifo}_{F}_{M}/'
savedata_path = f'/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/O3a-results-data/{Ifo}_{F}_{M}/'

if(os.path.isdir(savefig_path) == False):
    os.makedirs(savefig_path)
if(os.path.isdir(savedata_path) == False):
    os.makedirs(savedata_path)

print('Plots are stored in ',savefig_path )
print('Data are stored in ', savedata_path)

Plots are stored in  /home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/O3a-results-figs/HL_F_ER_M_Best/
Data are stored in  /home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/O3a-results-data/HL_F_ER_M_Best/


In [44]:
# create balanced dataset
dfDataset_filtered1 = dfnoise.sample(n=83528, replace=False, random_state=57) # Dataset for Test - Training
#dfDataset_filtered2 = dfDataset_filtered[~dfDataset_filtered.index.isin(dfDataset_filtered1.index)]
dfDataset_filtered2 = dfnoise.drop(dfDataset_filtered1.index) # Dataset for Validation
#dfDataset_filtered = dfDataset_filtered

In [45]:
# Create the final dataset 
dataset = pd.concat([dfDataset_filtered1, dfinj], ignore_index=True) # dataset in Training - Test 
print(dataset['label'].value_counts())

Noise    83528
Inj      83528
Name: label, dtype: int64


In [46]:
# Study the ranking statistics distribution
noise_hist = dataset[dataset['label'] == 'Noise']['amplitude']
inj_hist = dataset[dataset['label'] == 'Inj']['amplitude']

# Define common bin edges (example: 50 bins over combined range)
min_val = min(noise_hist.min(), inj_hist.min())
max_val = max(noise_hist.max(), inj_hist.max())
bins = np.linspace(min_val, max_val, 100)

# Create a new figure explicitly
plt.figure()

# Plot
sns.histplot(noise_hist, bins=bins, alpha=0.7, label='Noise', stat = 'count', element = 'step')
sns.histplot(inj_hist, bins=bins, alpha=0.5, label='Injections', stat = 'count', element = 'step')

plt.legend()
plt.yscale('log')
plt.xlabel('Ranking statistic', fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.title('Ranking statistic Noise - Injections Distributions', fontsize=15)

plt.savefig(savefig_path + 'amplitude_distribution.png', dpi=300, bbox_inches='tight')
plt.close()  # Close the figure to avoid accumulation

In [47]:
# create addiction features (if you need)
dataset['m_tot'] = dataset['m1'] + dataset['m2']
dataset['s_eff'] = (dataset['m1']*dataset['s1z'] + dataset['m2']*dataset['s2z']) / dataset['m_tot']
dataset['q'] = dataset['m2']/dataset['m1']

In [48]:
# Here we work only with Training - Test dataset

dfDataset = dataset[features_inj]
dfDataset['m_tot'] = dataset['m_tot']
dfDataset['s_eff'] = dataset['s_eff']
dfDataset['q'] = dataset['q']

dfDataset['L_ER'] = np.sqrt(1 - dfDataset['L_ERw']) + 0.3
dfDataset['H_ER'] = np.sqrt(1 - dfDataset['H_ERw']) + 0.3

dfDataset['dphi'] = dfDataset['H_phase'] - dfDataset['L_phase']
dfDataset['dt'] = dfDataset['Hend_time'] - dfDataset['Lend_time']
dfDataset['dD'] = dfDataset['H_effDist'] - dfDataset['L_effDist']
#dfDataset['mc_rotated'] = dfDataset['mc'] / dfDataset['m_tot']
#dfDataset['L_snr_rotated'] = dfDataset['L_snr'] / dfDataset['H_snr']

print(dfDataset['label'].value_counts())
print(dfDataset.columns)

Noise    83528
Inj      83528
Name: label, dtype: int64
Index(['L_snr', 'H_snr', 'V_snr', 'amplitude', 'L_autochi^2_PQ',
       'H_autochi^2_PQ', 'm1', 'm2', 'mc', 's1z', 's2z', 't_dur', 'nEvents',
       'L_ERw', 'H_ERw', 'L_phase', 'H_phase', 'L_effDist', 'H_effDist',
       'Lend_time', 'Hend_time', 'iFAR', 'label', 'gps_time', 'm_tot', 's_eff',
       'q', 'L_ER', 'H_ER', 'dphi', 'dt', 'dD'],
      dtype='object')


/tmp/ipykernel_2157616/182385904.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDataset['m_tot'] = dataset['m_tot']
/tmp/ipykernel_2157616/182385904.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDataset['s_eff'] = dataset['s_eff']
/tmp/ipykernel_2157616/182385904.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [49]:
#create the X matrix for features and y for the target  
X = dfDataset.drop(columns = ['label','gps_time','V_snr','m_tot', 's_eff',
                              'iFAR', 'amplitude', 'q', 'mc','L_ERw', 'H_ERw',
                              'L_phase', 'H_phase', 'L_effDist', 'H_effDist', 'Lend_time', 'Hend_time'], axis = 1)

#X.index.name = 'INDEX'
print('Training dataset:')
print(X.columns)
print('Labels array:')
y = pd.DataFrame(dfDataset['label'])
print(y.value_counts())

Training dataset:
Index(['L_snr', 'H_snr', 'L_autochi^2_PQ', 'H_autochi^2_PQ', 'm1', 'm2', 's1z',
       's2z', 't_dur', 'nEvents', 'L_ER', 'H_ER', 'dphi', 'dt', 'dD'],
      dtype='object')
Labels array:
label
Inj      83528
Noise    83528
dtype: int64


In [50]:
# split the test                                                                                                                           
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.3,
                                                    random_state=23
                                                    )


#create the dataframe to save the data
dfX_train = pd.DataFrame(X_train)
dfX_test = pd.DataFrame(X_test)
dfy_train = pd.DataFrame(y_train)
dfy_test = pd.DataFrame(y_test)
#add the index as column 
#dfX_train['INDEX'] = dfX_train.index
#dfX_test['INDEX'] = dfX_test.index
#dfy_train['INDEX'] = dfy_train.index
#dfy_test['INDEX'] = dfy_test.index
print(X)

           L_snr      H_snr  L_autochi^2_PQ  H_autochi^2_PQ         m1  \
0       5.017150   5.343976        0.775441        0.824745   3.269328   
1       5.153974   5.388456        0.884636        0.889501   2.640309   
2       5.616759   4.892799        0.602836        0.923775   6.068903   
3       5.723305   5.029689        0.822132        1.291943  15.561476   
4       4.852834   5.888259        0.944329        0.838534   1.021918   
...          ...        ...             ...             ...        ...   
167051  6.906860   7.906440        1.541581        0.881288   4.097899   
167052  4.857018   6.582831        0.905112        1.162780   5.036959   
167053  5.349629  12.605415        0.598217        1.651266   3.814512   
167054  7.573595   6.347766        1.315783        0.985199   6.361099   
167055  7.182309   7.312851        1.040693        0.797031   4.049705   

              m2       s1z       s2z       t_dur  nEvents      L_ER  H_ER  \
0       2.134979  0.353406  0.1970

In [14]:
# Uncomment this for grid search

"""
# grid search

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

# Define the Random Forest model
clf = RandomForestClassifier(random_state=42)

y_train_array = y_train.values.ravel()

# define the grid
param_grid = {
    'n_estimators': [15, 50, 100],
    'criterion': ['entropy', 'gini'],
    'max_depth': [10, 15, 20],
    'min_samples_leaf': [1, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt'],
}

# Define metrics function
precision = make_scorer(precision_score)
recall = make_scorer(recall_score)
f1 = make_scorer(f1_score, pos_label='Inj')

# Perform the grid search using F1 score
grid_search_f1 = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=f1, n_jobs=-1)
grid_search_f1.fit(X_train, y_train_array)

# Get the best parameters and scores
print("Best parameters (F1):", grid_search_f1.best_params_)
print("Best F1 score:", grid_search_f1.best_score_)

"""

'\n# grid search\n\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import make_scorer, precision_score, recall_score, f1_score\n\n# Define the Random Forest model\nclf = RandomForestClassifier(random_state=42)\n\ny_train_array = y_train.values.ravel()\n\n# define the grid\nparam_grid = {\n    \'n_estimators\': [15, 50, 100],\n    \'criterion\': [\'entropy\', \'gini\'],\n    \'max_depth\': [10, 15, 20],\n    \'min_samples_leaf\': [1, 5, 10],\n    \'min_samples_split\': [2, 5, 10],\n    \'max_features\': [\'sqrt\'],\n}\n\n# Define metrics function\nprecision = make_scorer(precision_score)\nrecall = make_scorer(recall_score)\nf1 = make_scorer(f1_score, pos_label=\'Inj\')\n\n# Perform the grid search using F1 score\ngrid_search_f1 = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=f1, n_jobs=-1)\ngrid_search_f1.fit(X_train, y_train_array)\n\n# Get the best parameters and scores\nprint("Best parameters (F1):", grid_search_f1.best_params_)\nprint("Best F1 sc

In [51]:
# Define and fit the classifier


clf = RandomForestClassifier(n_estimators = 100,
     #bootstrap=False, # added
     bootstrap=True,
     random_state=52,
     n_jobs =  -1,
     criterion =  'entropy',
     max_features =  'sqrt',
     max_depth =  12,
     #min_impurity_decrease=0.00005, # added
     min_samples_split= 5,
     min_samples_leaf = 1,
     #max_leaf_nodes =  8,
     #ccp_alpha=0.00005,
     #class weight
     #class_weight= {'Noise': 1, 'Inj': 2}
     )

clf.fit(X_train, y_train)
print('Probability classes: ',clf.classes_)                                                                    
y_pred_prob = clf.predict_proba(X_test)

#define the dataframe for the probabilities
dfprobs = pd.DataFrame(y_pred_prob, columns = ['ps', 'pTerr'])
dfprobs.index = X_test.index
print(dfprobs)

#create the dataframe to better manage the data
dfX_train = pd.DataFrame(X_train)
dfy_train = pd.DataFrame(y_train)
dfX_test = pd.DataFrame(X_test)
dfy_test = pd.DataFrame(y_test)

# Create the resulting dataset for test
dfy_pred_prob = pd.DataFrame(y_pred_prob)
test_dataset = pd.merge(dfX_test, dfy_test, left_index=True, right_index=True, how='inner')
#print(len(test_dataset))
#print(test_dataset['label'].value_counts())

# Add final informations
df_final_test = pd.merge(test_dataset, dfprobs, left_index=True, right_index=True, how='inner')
dfRFTriggers = df_final_test
dfRFTriggers['amplitude'] = dataset['amplitude']
dfRFTriggers['iFAR'] = dataset['iFAR']
dfRFTriggers['gps'] = dataset['gps_time']

# Separate noise and injection 
dfRFTriggers_inj = dfRFTriggers[dfRFTriggers['label'] == 'Inj']
dfRFTriggers_noise = dfRFTriggers[dfRFTriggers['label'] == 'Noise']

/tmp/ipykernel_2157616/876544492.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


Probability classes:  ['Inj' 'Noise']
              ps     pTerr
121175  0.621688  0.378312
59010   0.013909  0.986091
159773  0.999953  0.000047
28560   0.006659  0.993341
50708   0.039521  0.960479
...          ...       ...
85660   1.000000  0.000000
111046  0.935147  0.064853
107178  1.000000  0.000000
137192  0.904397  0.095603
110872  1.000000  0.000000

[50117 rows x 2 columns]


In [52]:
# Plot the ps distribution
# Get both data arrays
ps_noise = dfRFTriggers_noise['ps']
ps_inj = dfRFTriggers_inj['ps']

# Define common bins across both datasets
min_val = min(ps_noise.min(), ps_inj.min())
max_val = max(ps_noise.max(), ps_inj.max())
bins = np.linspace(min_val, max_val, 100)

plt.figure()

# Plot using consistent bins
sns.histplot(ps_noise, bins=bins, alpha=0.5, label='Noise', stat='count', element='step')
sns.histplot(ps_inj, bins=bins, label='Injections', stat='count', element='step')

# Formatting
plt.yscale('log')
plt.legend()
plt.xlabel(r'$p_s$', fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.title(r'$p_s$ Injections vs Noise Distribution', fontsize=15)
plt.savefig(savefig_path + 'ps_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

In [53]:
# Now we apply the logit transformation - ensure no inf values

# Ensure no inf values
dfRFTriggers_inj['ps_lim'] = np.log(dfRFTriggers_inj['ps'] / (1 - dfRFTriggers_inj['ps']))
dfRFTriggers_noise['ps_lim'] = np.log(dfRFTriggers_noise['ps'] / (1 - dfRFTriggers_noise['ps']))

# Replace inf and -inf with finite max/min values
m1 = dfRFTriggers_inj.loc[np.isfinite(dfRFTriggers_inj['ps_lim']), 'ps_lim'].max()
dfRFTriggers_inj['ps_lim'].replace([np.inf], m1, inplace=True)

m2 = dfRFTriggers_noise.loc[np.isfinite(dfRFTriggers_noise['ps_lim']), 'ps_lim'].max()
min2 = dfRFTriggers_noise.loc[np.isfinite(dfRFTriggers_noise['ps_lim']), 'ps_lim'].min()
dfRFTriggers_noise['ps_lim'].replace([np.inf], m2, inplace=True)
dfRFTriggers_noise['ps_lim'].replace([-np.inf], min2, inplace=True)

# Define common bins over both datasets
combined = pd.concat([dfRFTriggers_inj['ps_lim'], dfRFTriggers_noise['ps_lim']])
bins = np.linspace(combined.min(), combined.max(), 100)

plt.figure()

# Plot histograms
sns.histplot(dfRFTriggers_noise['ps_lim'], bins=bins, label='Noise', stat='count', element='step', alpha=0.5)
sns.histplot(dfRFTriggers_inj['ps_lim'], bins=bins, label='Injections', stat='count', element='step')

# Formatting
plt.yscale('log')
plt.xlabel(r'$\tilde{p}_s$')
plt.ylabel('Occurrencies')
plt.legend()
plt.title(r'Distribution of $\tilde{p}_s$')
plt.tight_layout()
plt.savefig(savefig_path + 'ps_lim_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

/tmp/ipykernel_2157616/2671188877.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRFTriggers_inj['ps_lim'] = np.log(dfRFTriggers_inj['ps'] / (1 - dfRFTriggers_inj['ps']))
/tmp/ipykernel_2157616/2671188877.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRFTriggers_noise['ps_lim'] = np.log(dfRFTriggers_noise['ps'] / (1 - dfRFTriggers_noise['ps']))
/tmp/ipykernel_2157616/2671188877.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [54]:
# Now compute the KDE for the pAstro

from sklearn.neighbors import KernelDensity

values_inj = dfRFTriggers_inj['ps_lim'].values.reshape(-1,1) # create trhe right format for the data

# Choose the binning
binning_noise = 0.6
binning_inj =0.8

kde_inj = KernelDensity(bandwidth=binning_inj, kernel='gaussian') # Define the kde with banwidth and kernel
kde_inj.fit(values_inj) # fit with the values

x = np.linspace(min(dfRFTriggers_inj['ps_lim']), max(dfRFTriggers_inj['ps_lim']), 25).reshape(-1,1) # take some random values from the data

# Estimate the density for the range of values
log_density_inj = kde_inj.score_samples(x) # Comput the log likelihood oe each sample under the model
density_inj = np.exp(log_density_inj)

values_noise = dfRFTriggers_noise['ps_lim'].values.reshape(-1,1) # create trhe right format for the data

kde_noise = KernelDensity(bandwidth=binning_noise, kernel='gaussian') # Define the kde with banwidth and kernel
kde_noise.fit(values_noise) # fit with the values


y = np.linspace(min(dfRFTriggers_noise['ps_lim']), max(dfRFTriggers_noise['ps_lim']), 25).reshape(-1,1) # take some random values from the data

# Estimate the density for the range of values
log_density_noise = kde_noise.score_samples(y) # Comput the log likelihood oe each sample under the model
density_noise = np.exp(log_density_noise)


In [55]:
# Plt the KDE

# Flatten KDE input arrays
x = x.flatten()
y = y.flatten()

# Combine both ps_lim arrays to define shared bin edges
ps_lim_inj = dfRFTriggers_inj['ps_lim']
ps_lim_noise = dfRFTriggers_noise['ps_lim']
combined = np.concatenate([ps_lim_inj, ps_lim_noise])
bins = np.linspace(combined.min(), combined.max(), 25)

# Create figure and twin axes
fig = plt.figure(figsize=(15, 10))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

# Plot histograms with shared bins and no auto-legend
sns.histplot(ps_lim_noise, bins=bins, alpha=0.3, ax=ax1, stat="density", color='b', label=None)
sns.histplot(ps_lim_inj, bins=bins, alpha=0.3, ax=ax1, stat="density", color='r', label=None)

# Plot KDEs (lines) with no auto-legend
sns.lineplot(x=x, y=density_inj, color='r', label=None)
sns.lineplot(x=y, y=density_noise, color='b', label=None)

# Manually define legend handles
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

custom_legend = [
    Line2D([0], [0], color='r', lw=2, label='Injections PDF'),
    Line2D([0], [0], color='b', lw=2, label='Noise PDF'),
    Patch(facecolor='r', alpha=0.3, label='Injections (hist)'),
    Patch(facecolor='b', alpha=0.3, label='Noise (hist)'),
]
# Ticks dimension
ax1.tick_params(axis='both', labelsize=25)  # ticks for ax1
ax2.tick_params(axis='both', labelsize=25)  # ticks for ax2

# Axis labels and title
ax1.set_xlabel(r'$\tilde{p}_s$',  fontsize=30)
ax1.set_ylabel('Histogram Density',  fontsize=30)
ax2.set_ylabel('PDF Density',  fontsize=30)
plt.title('PDF via KDE and Histogram',  fontsize=40)

# External single legend
ax1.legend(handles=custom_legend, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=20)

# Save and display
plt.tight_layout()
plt.savefig(savefig_path + 'pdf_final.png', dpi=300, bbox_inches='tight')
plt.close()

In [56]:
# Now compute the pAstro - Injections

values = np.array(dfRFTriggers_inj['ps_lim']) # read the ps_lim
ranks = values.reshape(-1, 1) # reshape it
#ranks_list = ranks.flatten().tolist() # flatten to a list

log_score_event = kde_inj.score_samples(ranks) # assign the score
score_event =  np.exp(log_score_event) # p(signal|ps_lim)

log_score_noise = kde_noise.score_samples(ranks)
score_noise = np.exp(log_score_noise) # p(noise|ps_lim)

# priors
Lambda1 = 36
Lambda0 = len(dfnoise)
print('Prior for Noise (RATE 6 months)', len(dfnoise))

# Now compute the pAstro from ps_lim

pAstro_rw_signals =[]
for rank, score_s, score_n in zip(ranks, score_event, score_noise):
   S = Lambda1*score_s
   N = Lambda0*score_n
   value = S/(S + N)
   pAstro_rw_signals.append(value.tolist())

dfRFTriggers_inj['pAstro_rw'] = pAstro_rw_signals

Prior for Noise (RATE 6 months) 173130


/tmp/ipykernel_2157616/1019813341.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRFTriggers_inj['pAstro_rw'] = pAstro_rw_signals


In [57]:
# Now compute the pAstro - Noise

values = np.array(dfRFTriggers_noise['ps_lim'])
ranks = values.reshape(-1, 1)

log_score_event = kde_inj.score_samples(ranks)
score_event =  np.exp(log_score_event)

log_score_noise = kde_noise.score_samples(ranks)
score_noise = np.exp(log_score_noise)

pAstro_rw_noise =[]
for rank, score_s, score_n in zip(ranks, score_event, score_noise):
   S = Lambda1*score_s
   N = Lambda0*score_n
   value = S/(S + N)
   pAstro_rw_noise.append(value.tolist())

dfscore_event_noise_triggers = pd.DataFrame(score_event)
dfscore_noise_noise_triggers = pd.DataFrame(score_noise)

dfRFTriggers_noise['pAstro_rw'] = pAstro_rw_noise

/tmp/ipykernel_2157616/3076381244.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRFTriggers_noise['pAstro_rw'] = pAstro_rw_noise


In [58]:
# Plot the result

plt.figure()
plt.scatter(dfRFTriggers_inj['amplitude'], dfRFTriggers_inj['pAstro_rw'], c = 'darkorange', s = 1, label = 'Injections')
plt.scatter(dfRFTriggers_noise['amplitude'], dfRFTriggers_noise['pAstro_rw'], c = 'b', s = 3, label = 'Noise')
plt.xlabel('Ranking statistic')
plt.ylabel(r'$p_\mathrm{astro}^{ps}$', fontsize = 15)
plt.title(r'$p_\mathrm{astro}^{ps}$ - ranking statistics', fontsize = 15)
plt.legend()
plt.savefig(savefig_path + 'pAstro_ps_amplitude_injections.png', dpi=300, bbox_inches='tight')
plt.legend()
plt.close()


In [59]:
# Define the the RoC function 
def RoC(df1, df2, column, var, label, save = True, thr=0.005, color = 'b'):
    if column not in df1.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame.")
# Drop missing values and sort unique values of the column                                                                                                       
    data1 = df1[column].dropna()
    data2 = df2[column].dropna()
    #thresholds = np.sort(data.unique())                                                                                                                             
    thresholds = np.arange(min(data1),max(data1), thr)

    # Calculate the cumulative distribution                                                                                                                          
    count_above_threshold1 = [np.sum(data1 > threshold) / len(data1) for threshold in thresholds]
    count_above_threshold2 =  [np.sum(data2 > threshold) / len(data2) for threshold in thresholds]

    # Create a DataFrame for the cumulative distribution                                                                                                             
    cumulative_df = pd.DataFrame({
        'threshold': thresholds,
        'count_above_threshold1': count_above_threshold1,
        'count_above_threshold2': count_above_threshold2
    })

    if save:
        cumulative_df.to_csv(path_data + f'/{var}_RoC_{label}.csv', index=False)

    # Plot the cumulative distribution                                                                                                                               
    plt.plot(cumulative_df['count_above_threshold1'], cumulative_df['count_above_threshold2'], label=label, c = color, linewidth=3.0)
    plt.yscale('log')
    plt.xscale('log')

    return cumulative_df

In [60]:
# Compute the Roc function  
plt.figure()
RoC(dfRFTriggers_noise, dfRFTriggers_inj, 'ps', 'Roc', r'$p_s$', thr = 0.0001, color = 'r', save = False)
RoC(dfRFTriggers_noise,dfRFTriggers_inj, 'amplitude', 'Roc', 'Ranking statistic', thr = 0.005, save = False, color= 'steelblue')
plt.legend()
plt.grid(True, which='both')
plt.ylabel('Nd (density)', fontsize=15)
plt.xlabel(r'$\alpha$', fontsize=15)
plt.savefig(savefig_path + 'RoC_O3a.png', dpi=300, bbox_inches='tight')
plt.close()
print(savefig_path)

/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/O3a-results-figs/HL_F_ER_M_Best/


In [61]:
# Now study the pAstro for events in the catalogue

events = dfEvents_confident#dfDataset_events_complete
events['label'] = 'event'

# fill in the added features (if you want to use them)
events['L_ER'] = np.sqrt(1 - events['L_ERw']) + 0.3
events['H_ER'] = np.sqrt(1 - events['H_ERw']) + 0.3
events['dphi'] = events['H_phase'] - events['L_phase']
events['dt'] = events['Hend_time'] - events['Lend_time']
events['dD'] = events['H_effDist'] - events['L_effDist']
events['m_tot'] = events['m1'] + events['m2']
events['s_eff'] = (events['m1']*events['s1z'] + events['m2']*events['s2z']) / events['m_tot']

y_evt = events['label'] 

In [63]:
X_event_test = events.drop(columns = ['Unnamed: 0','label','gps_time', 'mc', 'm_tot', 's_eff','V_snr',
                               'iFAR', 'amplitude',
                              'L_phase', 'H_phase', 'L_effDist', 'H_effDist', 'Lend_time', 'Hend_time','L_ERw', 'H_ERw',
                               'gps_time', 'commonName', 'GPS', 'pAstroMbta', 'GPS_int', 'gps_time_int'], axis = 1)


In [64]:
# Now apply the classifier
y_pred_prob_events = clf.predict_proba(X_event_test)
dfy_pred_prob_events = pd.DataFrame(y_pred_prob_events, columns = ['ps', 'pTerr'])

# Compactify the result
dfX_test_events = pd.DataFrame(X_event_test)
dfy_test_events = pd.DataFrame(y_evt)

test_dataset_events = pd.merge(dfX_test_events, dfy_test_events, left_index=True, right_index=True, how='inner')
dfRFTriggers_events= pd.merge(test_dataset_events, dfy_pred_prob_events, left_index=True, right_index=True, how='inner')

# fill in the dataframe
dfRFTriggers_events.index = dfEvents_confident.index #dfDataset_events_complete.index
dfRFTriggers_events['p_astro'] = dfEvents_confident['pAstroMbta']
dfRFTriggers_events['amplitude'] = dfEvents_confident['amplitude']
dfRFTriggers_events['iFAR'] = dfEvents_confident['iFAR']
dfRFTriggers_events['commonName'] = dfEvents_confident['commonName']

In [65]:
# Now compute the ps_lim for the events
dfRFTriggers_events['ps_lim'] = np.log(dfRFTriggers_events['ps'] / (1 - dfRFTriggers_events['ps']))

m1 = dfRFTriggers_events.loc[dfRFTriggers_events['ps_lim'] != np.inf, 'ps_lim'].max()
dfRFTriggers_events['ps_lim'] = dfRFTriggers_events['ps_lim'].replace([np.inf],m1)

# Compute pAstro for real Triggers

values = np.array(dfRFTriggers_events['ps_lim'])
ranks = values.reshape(-1, 1)
ranks_list = ranks.flatten().tolist()

log_score_event = kde_inj.score_samples(ranks)
score_event =  np.exp(log_score_event)


log_score_noise = kde_noise.score_samples(ranks)
score_noise = np.exp(log_score_noise)


Lambda1 = 36
#Lambda0 = len(noise)
print(len(score_event))
print(len(score_noise))
print(len(ranks_list))


pAstro_rw_signals =[]
for rank, score_s, score_n in zip(ranks, score_event, score_noise):
   S = Lambda1*score_s
   N = Lambda0*score_n
   value = S/(S + N)
   pAstro_rw_signals.append(value.tolist())


dfRFTriggers_events['pAstro_rw'] = pAstro_rw_signals
print(dfRFTriggers_events[['pAstro_rw', 'ps_lim']])

23
23
23
    pAstro_rw     ps_lim
0    1.000000  10.762128
1    1.000000  10.762128
2    1.000000  10.762128
3    1.000000  10.762128
4    1.000000   8.627789
5    1.000000  10.762128
6    1.000000  10.762128
7    1.000000  10.762128
8    1.000000  10.762128
9    1.000000  10.762128
10   0.040520   3.759374
11   1.000000  10.762128
12   1.000000   9.995415
13   0.786194   6.472326
14   1.000000  10.762128
15   1.000000  10.132898
16   1.000000  10.762128
17   1.000000  10.762128
18   1.000000  10.762128
19   0.084162   4.852095
20   1.000000  10.762128
21   0.098128   5.074166
22   0.041002   3.769493


In [66]:
# Plot the final result

plt.figure()
plt.scatter(dfRFTriggers_events['amplitude'], dfRFTriggers_events['pAstro_rw'], label = r'Events - $p_\mathrm{astro}^{ps}$', s = 55)
plt.scatter(dfRFTriggers_events['amplitude'], dfRFTriggers_events['p_astro'], label = r'Events - $p_\mathrm{astro}$', color = 'r', marker='x', s = 55)
plt.scatter(dfRFTriggers_events[dfRFTriggers_events['commonName'] == 'GW190924_021846']['amplitude'], dfRFTriggers_events[dfRFTriggers_events['commonName'] == 'GW190924_021846']['pAstro_rw'], label = 'GW190924_021846', color = 'g', s = 55)
plt.legend()
plt.title(r'$p_\mathrm{astro}^{ps}$ - $p_\mathrm{astro}$ - ranking statistics O3a Events')
plt.xlabel('ranking statistics')
plt.ylabel(r'$p_\mathrm{astro}$')
plt.ylim([-0.1,1.1])
# Plot a horizontal line at y = 25
plt.axhline(y=0.5, color='g', linestyle='-.')
plt.savefig(savefig_path + 'pAstro_ps_amplitude_O3aEvents_hihglight.png', dpi=300, bbox_inches='tight')
plt.close()

In [67]:
#--------------O3b consistency test
"""
path_data = '/home/lorenzo.mobilia/PhD-Thesis/RF-codes/data/O3b/'


NoiseHL_BBH = 'Triggers_noise_values_DT001_RealNoise_MbtaHL_Von_BBH.txt'
NoiseHL_BNS = 'Triggers_noise_values_DT001_RealNoise_MbtaHL_Von_BNS.txt'
NoiseHL_NSBH = 'Triggers_noise_values_DT001_RealNoise_MbtaHL_Von_NSBH.txt'
InjHL_BBH = 'Triggers_injection_values_DT001_RealNoise_BBH_MbtaHL_Von.txt'
InjHL_BNS = 'Triggers_injection_values_DT001_RealNoise_BNS_MbtaHL_Von.txt'
InjHL_NSBH = 'Triggers_injection_values_DT001_RealNoise_NSBH_MbtaHL_Von.txt'


noise_BBH = pd.read_csv(path_data + NoiseHL_BBH, delimiter='\t', index_col = False)
noise_BNS = pd.read_csv(path_data + NoiseHL_BNS, delimiter='\t', index_col = False)
noise_NSBH = pd.read_csv(path_data + NoiseHL_NSBH, delimiter='\t', index_col = False)

#.sample(n=83528)
noise = pd.concat([noise_BBH, noise_BNS, noise_NSBH], ignore_index=True)

injHL_BBH = pd.read_csv(path_data + InjHL_BBH, delimiter = '\t', index_col = False)
injHL_BNS = pd.read_csv(path_data + InjHL_BNS, delimiter = '\t', index_col = False)
injHL_NSBH = pd.read_csv(path_data + InjHL_NSBH, delimiter = '\t', index_col = False)

#inj = inj[inj['amplitude']<9]
noise['label'] = 'Noise'
injHL_BBH['label'] = 'Inj'
injHL_BNS['label'] = 'Inj'
injHL_NSBH['label'] = 'Inj'
"""
# upload the datasets  
Ifo = 'HL'
F = 'F0_ER_CONSISTENCY_TEST_O3b'
M = 'M_best' # result in paper are saved in M. - Good results, you fin in M_best - F0 the results with best model + ER. M_best + F0_noER best model + noER

#Define the features
features_inj = [#'index', 
                'L_snr', 'H_snr','V_snr',#F0
                'amplitude', #F9
                'L_autochi^2_PQ',	'H_autochi^2_PQ', #F0
                'm1', 'm2', #F1
                'mc',
                's1z', 's2z', #F2
                't_dur', #F3
                'nEvents', #F4
                'L_ERw', 'H_ERw', #F5
                'L_phase',	'H_phase', #F6
                'L_effDist', 'H_effDist', #F7
                'Lend_time', 'Hend_time',
                'iFAR',
                'label',
                'gps_time',
            ]

savefig_path = f'/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/O3a-results-figs/{Ifo}_{F}_{M}/'
savedata_path = f'/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/O3a-results-data/{Ifo}_{F}_{M}/'

if(os.path.isdir(savefig_path) == False):
    os.makedirs(savefig_path)
if(os.path.isdir(savedata_path) == False):
    os.makedirs(savedata_path)

print('Plots are stored in ',savefig_path )
print('Data are stored in ', savedata_path)


dfnoise = pd.read_csv('/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/data/O3b/noise_no_catalogue.csv', sep = ',')
dfinj = pd.read_csv('/home/lorenzo.mobilia/PhD-Thesis/pastro-o3-mbta-triggers-paper/RF-codes/data/O3b/injections.csv', sep = ',')

In [ ]:
# as above, clear the dataset

# Create the dataset for the Events
"""
noise = noise[features_inj]

dfEvents = pd.read_csv('/home/lorenzo.mobilia/PhD-Thesis/RF-codes/data/O3b-events/GWTC-3-confident.csv', delimiter = ';', usecols=['GPS', 'pAstroMbta', 'commonName'])
print('Events before cut in pAstro', len(dfEvents))
dfEvents = dfEvents.dropna(subset=['pAstroMbta'])
# Remove the events with pAstroMBTA == 0

dfEvents = dfEvents[dfEvents['pAstroMbta'] != 0]

dfEvents['GPS_int'] = dfEvents['GPS'].astype(int)
noise['gps_time_int'] = noise['gps_time'].astype(int)

print(dfEvents[dfEvents['commonName'] == 'GW200208_222617']['GPS_int'])
dfEvents.at[20, 'GPS_int'] = 1265235996
noise = noise.drop(100520)
print(noise[noise['gps_time_int'] == 1265235996])

# Remove those events from the dataset
dfDataset_filtered = noise[~noise['gps_time_int'].isin(dfEvents['GPS_int'])]

dfDataset_events = noise[noise['gps_time_int'].isin(dfEvents['GPS_int'])]
#row = noise[noise.index == 26937]
dfDataset_events_complete = dfDataset_events.merge(dfEvents, left_on='gps_time_int', right_on='GPS_int', how = 'inner')
#dfDataset_events_complete = pd.concat([dfDataset_events_complete,row], ignore_index = True) #Manually add the event GW190727_060333
dfDataset_events_pastro = dfDataset_events_complete['pAstroMbta']
dfDataset_events_complete = dfDataset_events_complete.drop_duplicates(subset=['commonName'])
#dfDataset_events_complete = dfDataset_events_complete.fillna('GW190727_060333')

#print(dfDataset_events_complete.head())
#print(dfDataset_events_complete.head())

# Now remove ALL the events present in the catalogue (GWTC2.1 Confident)

dfEvents_confident = pd.read_csv('/home/lorenzo.mobilia/PhD-Thesis/RF-codes/data/O3b-events/GWTC-3-confident.csv', delimiter = ';')
print('Events before cut in pAstro', len(dfEvents_confident))

dfEvents_confident['GPS_int'] = dfEvents_confident['GPS'].astype(int)
# Remove those events from the dataset
dfDataset_filtered = dfDataset_filtered[~dfDataset_filtered['gps_time_int'].isin(dfEvents_confident['GPS_int'])]
print('Final Dataset', dfDataset_filtered['label'].value_counts())
"""

In [68]:
# Concatenate the injections
"""
inj = pd.concat([injHL_BBH, injHL_BNS, injHL_NSBH])
# Drop the '||' column along with the Inj* columns
cols_to_drop = ['||', 'InjIndex', 'InjM1', 'InjM2', 'InjS1', 'InjS2', 'InjMc', 'InjDist', 'InjGps', 'Coinc', 't_index', 'rank_dphi', 'rank_effDistRatio', 'rank_dt', 'L_rank', 'H_rank']
inj = inj.drop(columns=cols_to_drop)
noise = dfDataset_filtered[features_inj]
"""

noise_balanced = dfnoise.sample(n=76389, replace=False)

# Create a balanced dataset 
dataset_O3b = pd.concat([noise_balanced, dfinj], ignore_index=True)
print(dataset_O3b['label'].value_counts())

dataset_O3b['m_tot'] = dataset_O3b['m1'] + dataset_O3b['m2']
dataset_O3b['s_eff'] = (dataset_O3b['m1']*dataset_O3b['s1z'] + dataset_O3b['m2']*dataset_O3b['s2z']) / dataset_O3b['m_tot']
dataset_O3b['q'] = dataset_O3b['m2']/dataset_O3b['m1']

dfDataset_O3b = dataset_O3b[features_inj]
dfDataset_O3b['m_tot'] = dataset_O3b['m_tot']
dfDataset_O3b['s_eff'] = dataset_O3b['s_eff']
dfDataset_O3b['q'] = dataset_O3b['q']

dfDataset_O3b['L_ER'] = np.sqrt(1 - dfDataset_O3b['L_ERw']) + 0.3
dfDataset_O3b['H_ER'] = np.sqrt(1 - dfDataset_O3b['H_ERw']) + 0.3

dfDataset_O3b['dphi'] = dfDataset_O3b['H_phase'] - dfDataset_O3b['L_phase']
dfDataset_O3b['dt'] = dfDataset_O3b['Hend_time'] - dfDataset_O3b['Lend_time']
dfDataset_O3b['dD'] = dfDataset_O3b['H_effDist'] - dfDataset_O3b['L_effDist']

print(dfDataset_O3b['label'].value_counts())

Noise    76389
Inj      76389
Name: label, dtype: int64
Noise    76389
Inj      76389
Name: label, dtype: int64


/tmp/ipykernel_2157616/1834712989.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDataset_O3b['m_tot'] = dataset_O3b['m_tot']
/tmp/ipykernel_2157616/1834712989.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDataset_O3b['s_eff'] = dataset_O3b['s_eff']
/tmp/ipykernel_2157616/1834712989.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
X_O3b = dfDataset.drop(columns = ['label','gps_time', 'm_tot', 'mc', 'q', 's_eff','V_snr',
                              'L_ERw', 'H_ERw', 'iFAR','amplitude',
                              'L_phase', 'H_phase', 'L_effDist', 'H_effDist', 'Lend_time', 'Hend_time'], axis = 1)

dfX_O3b = pd.DataFrame(X_O3b)
#X.index.name = 'INDEX'
print('Training dataset:')
print(X_O3b.columns)
print('Labels array:')
dfy_O3b = pd.DataFrame(dfDataset_O3b['label'])
#print(y.value_counts())

y_pred_prob_O3b = clf.predict_proba(X_O3b)
dfprobs_O3b = pd.DataFrame(y_pred_prob_O3b, columns = ['ps', 'pTerr'])

In [ ]:
dfy_pred_prob_O3b = pd.DataFrame(y_pred_prob_O3b)
test_dataset_O3b = pd.merge(dfX_O3b, dfy_O3b, left_index=True, right_index=True, how='inner')
df_final_test_O3b = pd.merge(test_dataset_O3b, dfprobs_O3b, left_index=True, right_index=True, how='inner')

dfRFTriggers_O3b = df_final_test_O3b
dfRFTriggers_O3b['amplitude'] = dataset_O3b['amplitude']
dfRFTriggers_O3b['iFAR'] = dataset_O3b['iFAR']
dfRFTriggers_O3b['gps'] = dataset_O3b['gps_time']

dfRFTriggers_inj_O3b = dfRFTriggers_O3b[dfRFTriggers_O3b['label'] == 'Inj']
dfRFTriggers_noise_O3b = dfRFTriggers_O3b[dfRFTriggers_O3b['label'] == 'Noise']

In [ ]:
plt.figure()
RoC(dfRFTriggers_noise_O3b, dfRFTriggers_inj_O3b, 'ps', 'Roc', r'$p_s$', thr = 0.0001, color = 'r', save = False)
RoC(dfRFTriggers_noise_O3b, dfRFTriggers_inj_O3b, 'amplitude', 'Roc', 'Ranking statistic', thr = 0.005, save = False, color= 'steelblue')
plt.legend()
plt.grid(True, which='both')
plt.ylabel('Nd (density)', fontsize=15)
plt.xlabel(r'$\alpha$', fontsize=15)
plt.savefig(savefig_path + 'RoC_O3b_O3aModel.png', dpi= 300,  bbox_inches = 'tight')
print(savefig_path)
plt.close()